In [36]:
import numpy as np
import re

roll_number = re.compile(r'([1,2][0,5,6,7,8,9].*)|tezansahu')

class Post(object):
    def __init__(self,author,subject,raw_text):
#         print(author,subject)
        self.author = re.findall(roll_number,author)[0]
        self.subject = re.findall(roll_number,subject)[0]
        self.raw_text = raw_text
        self.text,self.mentions = self.get_text_mentions(raw_text)
    
    def __str__(self):
        return f"{self.author} wrote the following about {self.subject} : \n {self.text}"
 
    def get_text_mentions(self,raw_text):
        # TODO
        return raw_text, None
    def asdict(self):
        return {"Author":self.author,"Subject":self.subject,"Text":self.text}


class Person(object):
    def __init__(self,id,name,degree,dept,hostel,ibs):
        self.id = re.findall(roll_number,id)[0]
        self.name = name
        self.degree = degree
        self.dept = dept
        self.hostel = hostel
        self.ibs = ibs   # TODO see how to put into sql
        self.posts_auth = []
        self.posts_subj = []
        self.posts_tagged = []
    def __str__(self):
        
        ibs_str = ", ".join(list(self.ibs))
        return f"Name: {self.name}, Program: {self.degree}, Dept: {self.dept}, Hostel: {self.hostel}, Clubs Associated with: {ibs_str}"
    
    def compute_post_stats(self):
        self.auth_len = np.mean([len(x.text.split()) for x in self.posts_auth])
        self.subj_len = np.mean([len(x.text.split()) for x in self.posts_subj])
        
    def asdict(self):
        return {"Name":self.name,"id":self.id,"Degree":self.degree,"Department":self.dept,"Hostel":self.hostel,
                "IBs":self.ibs,"Posts Auth":[x.asdict() for x in self.posts_auth],
                "Posts Subj":[x.asdict() for x in self.posts_subj]
               ,"Posts Tagged":[x.asdict() for x in self.posts_tagged]}

from bs4 import BeautifulSoup as soup

def get_pers(page,url):
    page = soup(page,parser="html.parser")
    personal_info = page.find("ul",class_="list-unstyled puser-details-list").find('li')

    name = personal_info.find('h2').text.split('\n')[0]
    print(name)
    dets = personal_info.find_all('h5')#.text

    try:
        degree,dept,hostel = dets[0].text.split('|')
    except:
        degree,dept,hostel = None,None,None
    try:
        ibs = (dets[1].text.split('|'))
    except:
        ibs = []

    new_pers = Person(url,name,degree,dept,hostel,ibs)

    incoming_edges = page.find("div",id="myTabContent").find_all("div",class_="sub-post-container")
    print(len(incoming_edges))
    for post in incoming_edges:
        parent = post.find_parent('div')
        if "tab-pane" not in parent["class"] or "impression" in parent["id"]:
            # This is done to make sure that only posts are taken in
            continue
        author = post.find("div",class_="user-details").find("a")
        author_id = author['href']
        author_name = author.text  # This is not strictly necessary, keeping it for now
        subject = author.find_next("div",class_="user-details").find("a")
        subject_id = subject['href']
        subject_name = subject.text

        post_text = post.find("p",class_="post-para").text
        new_post = Post(author_id,subject_id,post_text)
        if new_pers.id == new_post.subject:  # Handle the case of two people with the same name
            new_pers.posts_subj.append(new_post)
        elif new_pers.id == new_post.author:
            new_pers.posts_auth.append(new_post)

        else:   # Need to handle mentions separae
            new_pers.posts_tagged.append(new_post)

    new_pers.compute_post_stats()
    return new_pers
# print(new_pers.auth_len,new_pers.subj_len)

In [37]:
# In this cell I try to get the roll numbers of everyone from the xlsx
import pandas as pd
import requests
import json 

df = pd.read_excel('roll_list.xls',header=None)
roll_numbers = df.loc[:,1]
all_people = {}
for idx,roll in (enumerate(roll_numbers)):
#     if idx < 730:
#         continue

#     page = requests.get(url)
    try:
        roll = str(roll).strip()
        url = f"https://yearbook.sarc-iitb.org/student-{roll}/"
        page = open(f"raw_data/{roll}.html","r").read()
#     file = open(f"raw_data/{roll}.html","w")
#     file.write(page.text)
#     file.close()
#     pers = get_pers(page.text,url)
        pers = get_pers(page,url)
        all_people[roll] = (pers.asdict())
#         file = open("raw_data.jsonl","a")
#         print(json.dumps(pers.asdict()),file=file)
#         file.close()
    except Exception as e: 
        print(f"Skipping {roll} because {e}")

json.dump(all_people,open("raw_data.json","w"))


Meena Sourav Ramkishor 
0
Sachin Vasava 
1
Piyush Jirwankar [ aka Jeeru ]
5
Saieesh Jirwankar 
25
Tarang Jain 
27


/home/anshuln/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/anshuln/.local/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Harshal Kataria 
3
Dhruv Jain [ aka DeeJay ]
26
Gunjan Chitlange 
22
Rohit Vinodrao Tembhare 
7
Prajwal Vinodrao Bijwe [ aka Prajwal ]
8
Atharva Vijay Kanakdande 
4
Supriya Basappa Kamble 
4
Karthik Anantharaman 
3
Neilabh Banzal 
5
Trideep Das 
0
Prathamesh More 
1
Prince Rudranarayan Sharma 
2
Rushikesh Hiraman Zoting [ aka Rishi ]
30
Lahoti Shantanu Shyamkumar 
1
Karan Suresh Jagdale 
13
Shinde Kamlesh Dattatray 
10
Ajit Patwardhan 
18
Shyam Nitin Thombre 
2
Vaibhav Takale [ aka Takale ]
29
Aman Gopalrao Barahate 
0
Abhishek Meena [ aka Meena ]
4
Tushar Mishra 
35
Rajat Gupta 
1
Sanskriti Bhansali [ aka Sansii ]
64
Parag Khandelwal [ aka Paro ]
37
Lavanya Singh [ aka Lava ]
6
Sakshi Dayal 
60
Maharshi Pathak 
21
Yash Shrikant Maheshwary 
9
Harshvardhan Baldwa [ aka Harsh ]
8
Manoj Bhadu 
0
Komal Swami 
35
Sourav Rajak [ aka Rajak ]
5
Akshat Raj Lad 
83
Anand Singh 
4
Apoorv Som 
14
Nitish Tongia 
4
Kartikey Singh 
0
Shubham Kaushal 
0
Porapu Pavan Kumar 
1
Anudeep peela 
0
Rishi Aga

Hari Krishna Nama 
9
Rohan Abhishek Srikakulapu 
38
Vaddela Ashrut 
7
Chagam Dileep Kumar Reddy 
6
Sailendra Bathi Babu [ aka Bathi ]
18
Patnala Rohit Keerti Teja 
0
Bonela Mahith 
3
Sabbithi Naren Rahul [ aka Rahul ]
6
Govinda Siva Nagadev Lukka 
1
Ranjana B Kasangeri 
45
SATYABRATA NAIK [ aka Aytas,satyacse,ramurandom ]
19
Sreyas Raghavan 
2
Aditya Vavre [ aka Vavre ]
9
Vijaykrishna G 
30
Amanaganti Rohan Ganesh 
5
Sai Vishal Pothula 
16
Tarun Somavarapu 
3
Poorvi Hebbar [ aka Panda ]
99
Charan Narisimsetti 
14
Ramolla Nikhil Reddy 
3
Varshitha Mannem 
21
Diddi Sai Kiran 
13
Nikhil B 
10
Ramya Narayanasamy 
47
Satvik Ambati 
26
Tharun Teja Atla 
7
Penagamuri Pavan Chaitanya 
5
Aashish Waikar 
15
Himanshu Sheoran [ aka deuterium ]
61
Shashij Gupta 
25
Sarvesh Mehtani 
32
Ujjval Goury [ aka Uzzz ]
11
Rachit Bansal 
41
Manthan Jindal 
22
Sumit Sumit [ aka Sumit ]
58
Arvadia Kevin Dharmesh 
33
Rajesh Thorve 
55
Atharv Pawar [ aka Atharv ]
8
Syomantak Chaudhuri 
83
Anuj Diwan 
32
Manav Ba

Shubham Anand Jain 
59
Mithilesh Vaidya 
19
Naman Rajesh Narang 
2
Paras Bodke 
0
Prajval Nakrani 
5
Nikhil Arvind Bhaladhare 
0
Rishikesh Meshram 
39
Pragati Meshram 
28
Abhishek Sandeep Tanpure [ aka Abhi ]
15
Siddharth Chandak 
29
Rushil Heda 
2
Sakshee Anil Pimpale 
30
Sudhanshu Nimbalkar 
1
Karan Suresh Chate 
0
Prajwal Dnyaneshwar Kamble 
0
Sammed Mukesh Mangale 
8
Anubhav Agarwal 
31
Ritij Saini 
55
Tavish Mina [ aka Tavish ]
6
Harsh Sharma 
0
Sarthak Jain 
0
Bhavesh Garg 
8
Sayed Zafar Uzair 
2
Shivam Agarwal 
7
Ayush Raj Mehrotra [ aka Ayushraj ]
18
Nishant Goel 
4
Aditya Khanna 
10
Shreyas Goenka 
2
Divyansh Ahuja 
8
Yagya Mundra 
51
Pradeep Kadela [ aka Kadela ]
0
Tanmay Goyal 
0
Karan Amaliya 
0
Jaideep Kharadi 
2
Himanshi Mehta 
0
Sonal Kumar 
10
Nitish Kumar Das 
0
Mohd Safwan 
3
Abhijeet Verma 
2
Shailee Suryawanshi [ aka Jeni ]
85
Deepanshu Koshta 
0
Ritesh Botcha 
5
Etcherla Harshavardhan [ aka Harshavardhan ]
6
ARYAN LALL [ aka Aryan ]
13
Vemula Madhavan 
15
Agulla Su